### Overwrite normal function

In [43]:
class Test:
    def __init__(self):
        self.i = 4

    def __call__(self, *args, **kwargs):
        print('Call function')
        pass
    
    def func(self):
        print('func')

In [44]:
i = Test()
i()

Call function


In [45]:
def wrapper(f):
    print('Wrapper')
    def wrap(*args, **kwargs):
        print('before func')
        f(*args, **kwargs)
        print('after func')
    return wrap


In [46]:
i.func = wrapper(i.func)

Wrapper


In [47]:
i.func()

before func
func
after func


### Overwrite __call__
Have to overwrite class function as it is "special functon"

In [49]:
Test.__call__ = wrapper(Test.__call__)

Wrapper


In [50]:
i = Test()
i()

before func
Call function
after func


### Class decorator

In [143]:
def block(cls):
    print('Block')
    return cls

@block
class Test(object):
    def __init__(self):
        self.a = 1
        print('Init')
    

Block
{'__doc__': None, '__weakref__': <attribute '__weakref__' of 'Test' objects>, '__init__': <function Test.__init__ at 0x7facd6a201e0>, '__dict__': <attribute '__dict__' of 'Test' objects>, '__module__': '__main__'}


In [141]:
dut = Test()

Init


Probleem selles, et mul vaja peale init kutsumist muuta, ehk ikka metaclass vaja

In [164]:
class DBInterfaceMeta(type):
    def __init__(cls, name, bases, attrs, **kwargs):
        attr = getattr(cls, '__call__', None)
        if attr is None:
            print('WTF')
#         print(hasattr(cls, '__call__'))
        print('  Meta.__init__(cls=%s, name=%r, bases=%s, attrs=[%s], **%s)' % (
            cls, name, bases, ', '.join(attrs), kwargs
        ))
        return super().__init__(name, bases, attrs)
    
    # ran when instance is made
    def __call__(cls, *args, **kwargs):
        print('  Meta.__call__(cls=%s, args=%s, kwargs=%s)' % (cls, args, kwargs))
        return super().__call__(*args, **kwargs)
        
class Test(metaclass=DBInterfaceMeta):
    def __init__(self):
        self.a = 1
        print('MAIN Init')
        
    def __call__(self):
        print('CALL')

  Meta.__init__(cls=<class '__main__.Test'>, name='Test', bases=(), attrs=[__call__, __init__, __qualname__, __module__], **{})


In [155]:
dut = Test()

  Meta.__call__(cls=<class '__main__.Test'>, args=(), kwargs={})
MAIN Init


### Var to property

In [104]:
class RevealAccess(object):
    """A data descriptor that sets and returns values
       normally and prints a message logging their access.
    """

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        return self.val

    def __set__(self, obj, val):
        print('Updating', self.name)
        self.val = val

In [112]:
class MyClass(object):
    x = RevealAccess(10, 'var "x"')
    y = 5
    def __init__(self, val):
        self.lol = RevealAccess(10, 'var "x"')

m = MyClass(11)
m.lol

In [134]:
class Reg(object):
    def __init__(self, val):
        self._val_next = val
        self._val = val
    
    def __get__(self, obj, type=None):
        print('WTF')
        return self._val

    def __set__(self, obj, value):
        raise AttributeError()
    
class Cont(object):
    def __init__(self, val):
        self.reg = Reg(val)
#         self.reg = RevealAccess(10)
    
    def __getattribute__(self, name):
        attr = super(Cont, self).__getattribute__(name)
        if hasattr(attr, '__get__'):
            return attr.__get__(self, Cont)
        return attr
    
#     def setaatt

In [137]:
t = Cont(1)
t.reg
t.reg = 1

WTF


In [70]:
dut = Test()
dut.i

4

In [71]:
def getter(x):return 1

In [72]:
dut.i = property(getter)

In [80]:
dut.i(1)

TypeError: 'property' object is not callable

In [78]:
dir(property)

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']